In [1]:
#%%caption
!pip install mne
!pip install pybv

     |████████████████████████████████| 7.6 MB 2.3 MB/s eta 0:00:01
     |████████████████████████████████| 60 kB 12.1 MB/s eta 0:00:01


In [4]:
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import signal
import math
import mne
import pybv

In [268]:
# Leer el archivo CSV que contiene los nombres de las columnas
header_df = pd.read_csv('Data/csv/Header.csv', header = 0)
print(header_df)

sub = '04'  # Revisar el 52
# Leer el archivo CSV que contiene los datos
data_df = pd.read_csv('Data/csv/subject_'+sub+'.csv')
data_df.columns = header_df.columns
print(data_df.shape)

# Asignar el DataFrame que contiene los nombres de las columnas como encabezado del DataFrame que contiene los datos
df = pd.concat([data_df, header_df], axis = 0)

# Imprimir el DataFrame con el encabezado agregado
df

Empty DataFrame
Columns: [Time,  Fp1, Fp2, F5, AFZ, F6, T7, Cz, T8, P7, P3, PZ, P4, P8, O1, Oz, O2, Event, Target]
Index: []
(311098, 19)


,Time,Fp1,Fp2,F5,AFZ,F6,T7,Cz,T8,P7,P3,PZ,P4,P8,O1,Oz,O2,Event,Target
0,0.001953,-322.80,-334.250,237.820,-402.740,-646.86,-236.970,-8.4145,10.6160,198.030,-49.0750,-392.87,112.790,74.094,-16.9440,-125.74,-190.560,0,0
1,0.003906,-176.63,-329.620,241.850,-179.360,-654.73,-230.960,158.1500,7.6394,212.790,134.2200,139.97,188.290,67.972,58.9920,-106.92,-80.644,0,0
2,0.005859,-233.70,-390.110,179.260,-177.290,-682.12,-224.050,172.5500,-1.6537,179.460,152.0400,346.20,127.670,23.851,-5.8431,-114.47,-124.770,0,0
3,0.007812,-454.59,-478.640,91.292,-390.210,-702.42,-217.770,44.7880,-3.6419,120.370,3.8632,151.11,-31.501,-35.488,-181.1400,-133.39,-310.860,0,0
4,0.009766,-805.50,-604.100,-39.002,-762.840,-744.21,-228.900,-222.0500,-30.1460,18.812,-289.7700,-384.13,-279.010,-124.560,-439.6600,-188.80,-608.150,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311093,607.610000,-816.35,-61.784,-403.740,-758.100,287.12,-116.320,-796.1200,50.7670,-249.780,-639.1000,-1333.50,-754.940,20.586,-1435.7000,-597.72,-1224.300,0,0
311094,607.610000,-704.53,30.185,-250.540,-650.810,335.23,-100.750,-741.5900,71.5960,-158.270,-542.7700,-1256.10,-629.990,89.953,-1297.5000,-573.66,-1101.600,0,0
311095,607.610000,-554.77,130.930,-78.801,-409.980,374.03,-75.048,-588.0800,80.7160,-55.182,-388.8800,-928.98,-357.930,158.610,-1079.6000,-567.86,-908.500,0,0
311096,607.610000,-341.10,262.050,124.340,-58.906,429.12,-64.430,-338.0800,100.8700,82.412,-168.5000,-389.41,35.991,246.590,-789.7900,-564.13,-669.750,0,0


In [269]:
# Consultar el DataFrame para obtener las filas donde
df_filtradoN = df.query('Target == 1')
df_filtradoP = df.query('Target == 2')
arrayN = df_filtradoN.index.to_numpy()
arrayP = df_filtradoP.index.to_numpy()
# Imprimir el DataFrame filtrado
print(df_filtradoN.index.shape)

(640,)


In [270]:
# Crear un objeto Annotations para representar los eventos
onsetN = arrayN  # Tiempos de inicio de los eventos en muestras
durationN = 50*np.ones(len(df_filtradoN.index))  # Duración de los eventos en muestras
descriptionN = np.zeros(len(df_filtradoN.index))
tipoN = ["Stimulus"]* len(df_filtradoN)

onsetP = arrayP  # Tiempos de inicio de los eventos en muestras
durationP = 50*np.ones(len(df_filtradoP.index))  # Duración de los eventos en muestras
descriptionP = np.ones(len(df_filtradoP.index))
tipoP = ["Stimulus"]* len(df_filtradoP)

print(len(arrayN), len(durationN), len(descriptionN), len(tipoN) )
print(len(arrayP), len(durationP), len(descriptionP), len(tipoP) )

onset = np.concatenate([onsetN,onsetP], axis=0)
duration = np.concatenate([durationN,durationP],axis=0)
duration = duration.astype(int)
description = np.concatenate([descriptionN,descriptionP], axis=0)
description = description.astype(int)
tipo = tipoN+tipoP

events = []

#events = [onset, duration, description, tipo]
for i in range (len(onset)):
  if(description[i]==0):
    events.append({'onset': onset[i], 'duration': duration[i], 'description': 0, 'type': tipo[i]})
  elif(description[i]==1):
    events.append({'onset': onset[i], 'duration': duration[i], 'description': 1, 'type': tipo[i]})


for event in events:
  if event["type"] in ["Stimulus", "Response"]:
    if not isinstance(event["description"], int):
      print(event)

640 640 640 640
128 128 128 128


In [271]:
from pybv import write_brainvision

sfreq = 512

ch_names = ['Fp1', 'Fp2', 'F5', 'AFz', 'F6', 'T7', 'Cz', 'T8', 'P7', 'P3' , 'Pz', 'P4' , 'P8', 'O1', 'Oz', 'O2']
unit = ['µV']*16

data = df.iloc[:, [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]].to_numpy()
print(data.shape)

# for further parameters see our API documentation
write_brainvision(data=data.T, sfreq=sfreq, ch_names=ch_names,
                  folder_out="Data/pybv/",
                  fname_base="pybv_file"+sub,
                  events=events,
                  unit=unit,
                  overwrite=True)

(311098, 16)
